# Car Price Prediction - Final Optimized Model

Bu notebook, araç fiyat tahmini için optimize edilmiş bir LightGBM modeli içerir.

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest, f_regression
import lightgbm as lgb
import optuna
import warnings
warnings.filterwarnings('ignore')

## Veri Yükleme ve Hazırlama

In [3]:
# Load and prepare data
print("Loading and preparing data...")
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")

Loading and preparing data...
Train shape: (150826, 17)
Test shape: (37707, 16)


## Feature Engineering Fonksiyonu

In [4]:
# Enhanced Feature Engineering
def feature_engineering(df):
    df = df.copy()
    
    # Basic engineered features
    df['power_to_weight'] = df['horsepower'] / (df['km'] / 1000 + 1)
    df['age_km_interaction'] = df['age'] * df['km'] / 1000
    df['engine_power_ratio'] = df['horsepower'] / df['engine_liter']
    df['luxury_brand'] = df['brand'].isin(['Mercedes-Benz', 'BMW', 'Audi', 'Lexus', 'Porsche', 'Jaguar', 'Bentley', 'Rolls-Royce', 'Maserati']).astype(int)
    df['sports_car'] = df['model'].str.contains('M3|M4|M5|AMG|RS|GT|Turbo|Sport|Performance', case=False, na=False).astype(int)
    
    # Advanced features
    df['year_squared'] = df['year'] ** 2
    df['km_log'] = np.log1p(df['km'])
    df['hp_per_year'] = df['horsepower'] / (2024 - df['year'] + 1)
    df['premium_features'] = (df['luxury_brand'] + df['sports_car']).clip(0, 1)
    df['age_hp_ratio'] = df['age'] / (df['horsepower'] + 1)
    df['km_per_year'] = df['km'] / (df['age'] + 1)
    df['engine_efficiency'] = df['horsepower'] / (df['engine_liter'] * df['age'] + 1)
    
    # Price-related features based on domain knowledge
    df['depreciation_factor'] = np.exp(-0.1 * df['age'])  # Exponential depreciation
    df['mileage_penalty'] = np.exp(-df['km'] / 100000)  # Mileage penalty
    df['luxury_premium'] = df['luxury_brand'] * df['horsepower'] / 100
    
    # Categorical binning
    df['age_group'] = pd.cut(df['age'], bins=[0, 3, 7, 15, 100], labels=[0, 1, 2, 3])
    df['km_group'] = pd.cut(df['km'], bins=[0, 30000, 80000, 150000, 500000], labels=[0, 1, 2, 3])
    df['hp_group'] = pd.cut(df['horsepower'], bins=[0, 200, 300, 450, 2000], labels=[0, 1, 2, 3])
    
    return df

## Feature Hazırlama Fonksiyonu

In [5]:
# Prepare features with enhanced encoding
def prepare_features(train_df, test_df):
    categorical_cols = ['brand', 'fuel_type', 'transmission', 'accident', 'clean_title', 'color_ext', 'color_int']
    
    # Target encoding for high-cardinality categorical variables
    target_encoded_cols = ['brand', 'color_ext', 'color_int']
    
    for col in target_encoded_cols:
        # Calculate mean target for each category
        mean_target = train_df.groupby(col)['price'].mean()
        # Apply smoothing to avoid overfitting
        global_mean = train_df['price'].mean()
        smoothing = 100
        counts = train_df.groupby(col).size()
        smoothed_means = (counts * mean_target + smoothing * global_mean) / (counts + smoothing)
        
        # Apply to train and test
        train_df[col + '_target_encoded'] = train_df[col].map(smoothed_means).fillna(global_mean)
        test_df[col + '_target_encoded'] = test_df[col].map(smoothed_means).fillna(global_mean)
    
    # Label encoding for other categorical variables
    label_encoded_cols = ['fuel_type', 'transmission', 'accident', 'clean_title']
    for col in label_encoded_cols:
        le = LabelEncoder()
        combined_values = pd.concat([train_df[col], test_df[col]]).astype(str)
        le.fit(combined_values)
        train_df[col + '_encoded'] = le.transform(train_df[col].astype(str))
        test_df[col + '_encoded'] = le.transform(test_df[col].astype(str))
    
    # Select all features
    feature_cols = (['year', 'km', 'horsepower', 'engine_liter', 'age', 
                    'horsepower_missing', 'engine_liter_missing',
                    'power_to_weight', 'age_km_interaction', 'engine_power_ratio',
                    'luxury_brand', 'sports_car', 'year_squared', 'km_log', 
                    'hp_per_year', 'premium_features', 'age_hp_ratio', 'km_per_year',
                    'engine_efficiency', 'depreciation_factor', 'mileage_penalty', 'luxury_premium',
                    'age_group', 'km_group', 'hp_group'] + 
                   [col + '_target_encoded' for col in target_encoded_cols] +
                   [col + '_encoded' for col in label_encoded_cols])
    
    X_train = train_df[feature_cols]
    y_train = train_df['price']
    X_test = test_df[feature_cols]
    
    return X_train, y_train, X_test, feature_cols

## Feature Engineering Uygulama

In [6]:
train_fe = feature_engineering(train)
test_fe = feature_engineering(test)
X_train, y_train, X_test, feature_cols = prepare_features(train_fe, test_fe)

print(f"Data prepared. Training shape: {X_train.shape}")

Data prepared. Training shape: (150826, 32)


## Feature Selection

In [7]:
# Feature selection
print("Performing feature selection...")
selector = SelectKBest(score_func=f_regression, k=25)  # Select top 25 features
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

selected_features = [feature_cols[i] for i in selector.get_support(indices=True)]
print(f"Selected {len(selected_features)} features")

# Convert back to DataFrame for easier handling
X_train_selected = pd.DataFrame(X_train_selected, columns=selected_features)
X_test_selected = pd.DataFrame(X_test_selected, columns=selected_features)

Performing feature selection...
Selected 25 features


## Hyperparameter Optimization

In [8]:
# Advanced LightGBM optimization
def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 50, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 200),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 20),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 20),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_split_gain': trial.suggest_float('min_split_gain', 0, 10),
        'random_state': 42,
        'verbosity': -1,
        'force_col_wise': True
    }
    
    # 3-fold CV for faster optimization
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    cv_scores = []
    
    for train_idx, val_idx in kf.split(X_train_selected):
        X_tr, X_val = X_train_selected.iloc[train_idx], X_train_selected.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        train_data = lgb.Dataset(X_tr, label=y_tr)
        val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
        
        model = lgb.train(
            params,
            train_data,
            valid_sets=[val_data],
            num_boost_round=1500,
            callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)]
        )
        
        y_pred = model.predict(X_val, num_iteration=model.best_iteration)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        cv_scores.append(rmse)
    
    return np.mean(cv_scores)

In [9]:
print("\nStarting advanced hyperparameter optimization...")
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)  # More trials for better optimization

print(f"\nBest CV RMSE: {study.best_value:.2f}")
print("Best parameters:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

[I 2025-05-31 11:14:51,907] A new study created in memory with name: no-name-2af6f59f-7a0a-4900-b31a-cc8c386d781c



Starting advanced hyperparameter optimization...
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[133]	valid_0's rmse: 74948.6
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[103]	valid_0's rmse: 73886.5
Training until validation scores don't improve for 100 rounds


[I 2025-05-31 11:14:53,462] Trial 0 finished with value: 73982.63898284589 and parameters: {'num_leaves': 472, 'learning_rate': 0.07195650266184664, 'feature_fraction': 0.9071233927475909, 'bagging_fraction': 0.8456506145047082, 'bagging_freq': 1, 'min_child_samples': 32, 'reg_alpha': 8.22249127721598, 'reg_lambda': 19.003893419403045, 'max_depth': 3, 'min_split_gain': 6.89292511013125}. Best is trial 0 with value: 73982.63898284589.


Early stopping, best iteration is:
[96]	valid_0's rmse: 73112.8
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[22]	valid_0's rmse: 75400.5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[24]	valid_0's rmse: 74162.4
Training until validation scores don't improve for 100 rounds


[I 2025-05-31 11:14:57,545] Trial 1 finished with value: 74401.81537440495 and parameters: {'num_leaves': 462, 'learning_rate': 0.10389840214239383, 'feature_fraction': 0.7133115491714345, 'bagging_fraction': 0.8293290830398392, 'bagging_freq': 9, 'min_child_samples': 12, 'reg_alpha': 3.049273136658077, 'reg_lambda': 9.198486222125714, 'max_depth': 12, 'min_split_gain': 2.1999007334966256}. Best is trial 0 with value: 73982.63898284589.


...

Early stopping, best iteration is:
[114]	valid_0's rmse: 73078.8
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[259]	valid_0's rmse: 74823.7
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[257]	valid_0's rmse: 73726.9
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[342]	valid_0's rmse: 73029.7


[I 2025-05-31 11:24:13,845] Trial 98 finished with value: 73860.06944363087 and parameters: {'num_leaves': 376, 'learning_rate': 0.014292187254636053, 'feature_fraction': 0.732628507347585, 'bagging_fraction': 0.7426978087810651, 'bagging_freq': 6, 'min_child_samples': 168, 'reg_alpha': 1.9693655347061243, 'reg_lambda': 1.029983508873814, 'max_depth': 9, 'min_split_gain': 8.355650876264525}. Best is trial 72 with value: 73831.88672420561.


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[295]	valid_0's rmse: 74810.7
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[183]	valid_0's rmse: 73761.9
Training until validation scores don't improve for 100 rounds


[I 2025-05-31 11:24:18,266] Trial 99 finished with value: 73877.26943644659 and parameters: {'num_leaves': 437, 'learning_rate': 0.02767300855206974, 'feature_fraction': 0.7726082784089833, 'bagging_fraction': 0.6667108192691783, 'bagging_freq': 7, 'min_child_samples': 186, 'reg_alpha': 3.5014717999676144, 'reg_lambda': 7.458482493918028, 'max_depth': 8, 'min_split_gain': 7.565500069471457}. Best is trial 72 with value: 73831.88672420561.


Early stopping, best iteration is:
[159]	valid_0's rmse: 73059.2

Best CV RMSE: 73831.89
Best parameters:
  num_leaves: 403
  learning_rate: 0.010207244346730912
  feature_fraction: 0.682420026352462
  bagging_fraction: 0.7097300159161973
  bagging_freq: 8
  min_child_samples: 192
  reg_alpha: 1.711919753246828
  reg_lambda: 6.036732986068603
  max_depth: 8
  min_split_gain: 7.020385068244367


## Final Model Training

In [10]:
# Train final model with 5-fold CV
print("\nTraining final model with 5-fold CV...")
best_params = study.best_params
best_params.update({
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'random_state': 42,
    'verbosity': -1,
    'force_col_wise': True
})

kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros(len(X_train_selected))
test_preds = np.zeros(len(X_test_selected))

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_selected)):
    print(f"Training fold {fold + 1}...")
    X_tr, X_val = X_train_selected.iloc[train_idx], X_train_selected.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    train_data = lgb.Dataset(X_tr, label=y_tr)
    val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
    
    model = lgb.train(
        best_params,
        train_data,
        valid_sets=[val_data],
        num_boost_round=2000,
        callbacks=[lgb.early_stopping(150), lgb.log_evaluation(0)]
    )
    
    oof_preds[val_idx] = model.predict(X_val, num_iteration=model.best_iteration)
    test_preds += model.predict(X_test_selected, num_iteration=model.best_iteration) / kf.n_splits


Training final model with 5-fold CV...
Training fold 1...
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[423]	valid_0's rmse: 69930.6
Training fold 2...
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[494]	valid_0's rmse: 78708.5
Training fold 3...
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[507]	valid_0's rmse: 77464.4
Training fold 4...
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[630]	valid_0's rmse: 68799.5
Training fold 5...
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[440]	valid_0's rmse: 73979.2


## Post-processing ve Submission

In [11]:
# Post-processing
test_preds = np.maximum(test_preds, 1000)  # Minimum price

final_rmse = np.sqrt(mean_squared_error(y_train, oof_preds))
print(f"\nFinal CV RMSE: {final_rmse:.2f}")

# Create submission
submission = pd.DataFrame({
    'id': test['id'],
    'price': test_preds
})

submission.to_csv('final_optimized_submission.csv', index=False)
print(f"\nFinal optimized submission saved!")
print(f"Predicted prices range: {test_preds.min():.2f} - {test_preds.max():.2f}")
print(f"Mean predicted price: {test_preds.mean():.2f}")

print(f"\nSelected features: {selected_features}")


Final CV RMSE: 73881.43

Final optimized submission saved!
Predicted prices range: 8602.21 - 266081.33
Mean predicted price: 44124.64

Selected features: ['year', 'km', 'horsepower', 'engine_liter', 'age', 'power_to_weight', 'age_km_interaction', 'engine_power_ratio', 'year_squared', 'km_log', 'hp_per_year', 'age_hp_ratio', 'km_per_year', 'engine_efficiency', 'depreciation_factor', 'mileage_penalty', 'luxury_premium', 'age_group', 'km_group', 'hp_group', 'brand_target_encoded', 'color_ext_target_encoded', 'color_int_target_encoded', 'accident_encoded', 'clean_title_encoded']


## Sonuçları Görüntüleme

In [12]:
# Display submission preview
print("\nSubmission preview:")
print(submission.head(10))

print(f"\nSubmission statistics:")
print(submission['price'].describe())


Submission preview:
   id          price
0   0    9631.083364
1   6   28054.572409
2  11   27406.609086
3  12   33130.645254
4  20  129015.759903
5  22   12763.897270
6  31   63698.759571
7  33   13033.640464
8  34   43853.538367
9  35   19824.126868

Submission statistics:
count     37707.000000
mean      44124.643377
std       30195.467545
min        8602.208957
25%       20702.248696
50%       37499.181618
75%       59203.539184
max      266081.331151
Name: price, dtype: float64
